# 1.Data import and cleansing

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
import statsmodels.api as sm
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier

In [2]:
ca2 = pd.read_csv('banking.csv', header=0)
ca2 = ca2.dropna()
print(ca2.shape)
print(list(ca2.columns))

(41188, 21)
['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp_var_rate', 'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed', 'y']


In [3]:
cadrop=ca2[['contact', 'month', 'day_of_week', 'duration', 'campaign']]
cadrop

,contact,month,day_of_week,duration,campaign
0,cellular,aug,thu,210,1
1,cellular,nov,fri,138,1
2,cellular,jun,thu,339,3
3,cellular,apr,fri,185,2
4,cellular,aug,fri,137,1
...,...,...,...,...,...
41183,telephone,jun,thu,222,1
41184,telephone,may,thu,196,2
41185,telephone,may,wed,62,3
41186,telephone,oct,tue,200,2


In [4]:
ca= ca2.drop(['contact', 'month', 'day_of_week', 'duration', 'campaign'],axis=1)
ca

,age,job,marital,education,default,housing,loan,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,44,blue-collar,married,basic.4y,unknown,yes,no,999,0,nonexistent,1.4,93.444,-36.1,4.963,5228.1,0
1,53,technician,married,unknown,no,no,no,999,0,nonexistent,-0.1,93.200,-42.0,4.021,5195.8,0
2,28,management,single,university.degree,no,yes,no,6,2,success,-1.7,94.055,-39.8,0.729,4991.6,1
3,39,services,married,high.school,no,no,no,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,0
4,55,retired,married,basic.4y,no,yes,no,3,1,success,-2.9,92.201,-31.4,0.869,5076.2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,59,retired,married,high.school,unknown,no,yes,999,0,nonexistent,1.4,94.465,-41.8,4.866,5228.1,0
41184,31,housemaid,married,basic.4y,unknown,no,no,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,0
41185,42,admin.,single,university.degree,unknown,yes,yes,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
41186,48,technician,married,professional.course,no,no,yes,999,0,nonexistent,-3.4,92.431,-26.9,0.742,5017.5,0


In [5]:
ca.dtypes

age                 int64
job                object
marital            object
education          object
default            object
housing            object
loan               object
pdays               int64
previous            int64
poutcome           object
emp_var_rate      float64
cons_price_idx    float64
cons_conf_idx     float64
euribor3m         float64
nr_employed       float64
y                   int64
dtype: object

In [6]:
base = ca[['y']]

In [7]:
## identify categorical variables
cat = ca.select_dtypes(exclude=['int', 'float']).columns
print(len(cat))
print('Number of categoriacal features:', len(cat))

CatVar=ca[cat]
Nl=[]
for column in CatVar.columns:
    if CatVar[column].isnull().values.any() == True:
        Nl.append(column)
    
print('columns with missing values:', Nl)

7
Number of categoriacal features: 7
columns with missing values: []


In [8]:
for column in CatVar.columns:
    print(CatVar.groupby(column)[column].agg(['count']))

               count
job                 
admin.         10422
blue-collar     9254
entrepreneur    1456
housemaid       1060
management      2924
retired         1720
self-employed   1421
services        3969
student          875
technician      6743
unemployed      1014
unknown          330
          count
marital        
divorced   4612
married   24928
single    11568
unknown      80
                     count
education                 
basic.4y              4176
basic.6y              2292
basic.9y              6045
high.school           9515
illiterate              18
professional.course   5243
university.degree    12168
unknown               1731
         count
default       
no       32588
unknown   8597
yes          3
         count
housing       
no       18622
unknown    990
yes      21576
         count
loan          
no       33950
unknown    990
yes       6248
             count
poutcome          
failure       4252
nonexistent  35563
success       1373


In [9]:
CatVar['education']=np.where(CatVar['education']=='basic.4y', 'basic',
                            np.where(CatVar['education']=='basic.6y', 'basic',
                                    np.where(CatVar['education']=='basic.9y', 'basic',
                                            np.where(CatVar['education']=='illiterate', 'unknown',
                                                    CatVar['education']))))

CatVar['job']=np.where(CatVar['job']=='entrepreneur', 'management',
                      np.where(CatVar['job']=='self-employed', 'management',
                              np.where(CatVar['job']=='retired', 'other 1', 
                                       np.where(CatVar['job']=='unknown', 'other 2', 
                                                np.where(CatVar['job']=='student', 'other 1', 
                                                         np.where(CatVar['job']=='unemployed', 'other 2', 
                                                                  np.where(CatVar['job']=='housemaid', 'other 2', 
                                                                           CatVar['job'])))))))


CatVar['marital']=np.where(CatVar['marital']=='unknown','single',CatVar['marital'])


CatVar['default']=np.where(CatVar['default']=='yes','unknown',CatVar['default'])

/Users/szymanskia1/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/szymanskia1/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/szymanskia1/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

In [10]:
for column in CatVar.columns:
    print(CatVar.groupby(column)[column].agg(['count']))

             count
job               
admin.       10422
blue-collar   9254
management    5801
other 1       2595
other 2       2404
services      3969
technician    6743
          count
marital        
divorced   4612
married   24928
single    11648
                     count
education                 
basic                12513
high.school           9515
professional.course   5243
university.degree    12168
unknown               1749
         count
default       
no       32588
unknown   8600
         count
housing       
no       18622
unknown    990
yes      21576
         count
loan          
no       33950
unknown    990
yes       6248
             count
poutcome          
failure       4252
nonexistent  35563
success       1373


In [11]:
Cat_dummies = pd.get_dummies(CatVar, drop_first=True)
print(CatVar.shape)
print(Cat_dummies.shape)

(41188, 7)
(41188, 19)


In [12]:
## Identify continuous variables
cont1 = ca.select_dtypes(include=['int', 'float']).columns
print('initial number of continuous features: ',len(cont1))


## Exclude those previously classified as dummies
cont4=set(cont1).difference(cat)
cont=set(cont4).difference(base)
print('initial number of continuous features after exclusions: ',len(cont))
print()

ContVar= ca[cont]

Nl3=[]
for column in ContVar.columns:
    if ContVar[column].isnull().values.any() == True:
        Nl3.append(column)
    
print('features with missing values: ',Nl3)

initial number of continuous features:  9
initial number of continuous features after exclusions:  8

features with missing values:  []


### Standardise continuous variables

In [13]:
from sklearn.preprocessing import StandardScaler

### Standardise variables 
sc=StandardScaler()

sc.fit(ContVar)
ContVar_std=pd.DataFrame(sc.transform(ContVar),columns=cont)

### double check for missing values
Nl4=[]
for column in ContVar_std.columns:
    if ContVar_std[column].isnull().values.any() == True:
        Nl4.append(column)
    
print('Variables with missing in ContVar_std: ',Nl4)

Variables with missing in ContVar_std:  []


In [14]:
print(base.shape)
print(ContVar.shape)
print(ContVar_std.shape)
print(Cat_dummies.shape)

(41188, 1)
(41188, 8)
(41188, 8)
(41188, 19)


In [15]:
#df = pd.concat([base,ContVar,Cat_dummies,],axis=1)
df = pd.concat([base,ContVar_std,Cat_dummies],axis=1)
df.shape
df.to_csv('banking_cleansed.csv', index=False) # make sure you drop index when saving to csv
print(df.shape)

(41188, 28)


# 2. Models

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
df=pd.read_csv('banking_cleansed.csv')
X = df.iloc[:,1:]
y = df.iloc[:,0]
os = SMOTE(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
columns = X_train.columns
os_data_X,os_data_y=os.fit_sample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=['y'])

Using TensorFlow backend.


In [17]:
df.head(5)

,y,previous,euribor3m,cons_price_idx,nr_employed,cons_conf_idx,emp_var_rate,pdays,age,job_blue-collar,...,education_professional.course,education_university.degree,education_unknown,default_unknown,housing_unknown,housing_yes,loan_unknown,loan_yes,poutcome_nonexistent,poutcome_success
0,0,-0.349494,0.773575,-0.227465,0.845170,0.951267,0.839061,0.195414,0.381527,1,...,0,0,0,1,0,1,0,0,1,0
1,0,-0.349494,0.230456,-0.649003,0.398115,-0.323542,-0.115781,0.195414,1.245157,0,...,0,0,1,0,0,0,0,0,1,0
2,1,3.691766,-1.667578,0.828107,-2.428157,0.151810,-1.134279,-5.117342,-1.153816,0,...,0,1,0,0,0,1,0,0,0,1
3,0,-0.349494,-1.277824,-0.864955,-0.940281,-1.425496,-1.197935,0.195414,-0.098268,0,...,0,0,0,0,0,0,0,0,1,0
4,1,1.671136,-1.586859,-2.374889,-1.257233,1.966794,-1.898153,-5.133393,1.437075,0,...,0,0,0,0,0,1,0,0,0,1


## 2.1 Decision Tree

In [18]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import roc_auc_score
dt = DecisionTreeClassifier(criterion='gini', min_samples_split=200,min_samples_leaf=100, max_depth=3)
dt.fit(os_data_X, os_data_y)
y_pred3 = dt.predict(X_test)
print('Misclassified samples: %d' % (y_test != y_pred3).sum())
print(metrics.classification_report(y_test, y_pred3))
print (roc_auc_score(y_test, y_pred3))

Misclassified samples: 2028
              precision    recall  f1-score   support

           0       0.95      0.87      0.90     10981
           1       0.36      0.60      0.45      1376

    accuracy                           0.84     12357
   macro avg       0.65      0.73      0.68     12357
weighted avg       0.88      0.84      0.85     12357

0.7341186441485611


## 2.2 Create Node variable

In [19]:
from sklearn.tree import _tree

nn=0
def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    #print ("def tree({}):" .format(", " .join(feature_names)))

    def recurse(node, depth):
        indent = "    " * depth

        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            table = 'X_train'
            name = table+"['"+feature_name[node]+"']"
            

            threshold = tree_.threshold[node]
            print ("{}if {} <= {}:".format(indent, name, threshold))
            recurse(tree_.children_left[node], depth + 1)
            print ("{}else:  # if {} > {}".format(indent, name, threshold))
            recurse(tree_.children_right[node], depth + 1)
        else:
            def increment():
                global nn
                nn=nn+1
            increment()
            print ("{}return 'Node_{}'".format(indent, nn))

    recurse(0, 1)

In [20]:
tree_to_code(dt,columns)

    if X_train['nr_employed'] <= -0.9410539865493774:
        if X_train['pdays'] <= -5.0745530128479:
            if X_train['cons_price_idx'] <= -2.3744064569473267:
                return 'Node_1'
            else:  # if X_train['cons_price_idx'] > -2.3744064569473267
                return 'Node_2'
        else:  # if X_train['pdays'] > -5.0745530128479
            if X_train['loan_yes'] <= 0.5:
                return 'Node_3'
            else:  # if X_train['loan_yes'] > 0.5
                return 'Node_4'
    else:  # if X_train['nr_employed'] > -0.9410539865493774
        if X_train['cons_conf_idx'] <= -1.231077253818512:
            if X_train['cons_price_idx'] <= -0.8649553954601288:
                return 'Node_5'
            else:  # if X_train['cons_price_idx'] > -0.8649553954601288
                return 'Node_6'
        else:  # if X_train['cons_conf_idx'] > -1.231077253818512
            if X_train['default_unknown'] <= 0.5:
                return 'Node_7'
            el

In [21]:
def tree(X_train):
    if X_train['nr_employed'] <= -0.9410539865493774:
        if X_train['pdays'] <= -5.0745530128479:
            if X_train['cons_price_idx'] <= -2.3744064569473267:
                return 'Node_1'
            else:  # if X_train['cons_price_idx'] > -2.3744064569473267
                return 'Node_2'
        else:  # if X_train['pdays'] > -5.0745530128479
            if X_train['loan_yes'] <= 0.5:
                return 'Node_3'
            else:  # if X_train['loan_yes'] > 0.5
                return 'Node_4'
    else:  # if X_train['nr_employed'] > -0.9410539865493774
        if X_train['cons_conf_idx'] <= -1.231077253818512:
            if X_train['cons_price_idx'] <= -0.8649553954601288:
                return 'Node_5'
            else:  # if X_train['cons_price_idx'] > -0.8649553954601288
                return 'Node_6'
        else:  # if X_train['cons_conf_idx'] > -1.231077253818512
            if X_train['default_unknown'] <= 0.5:
                return 'Node_7'
            else:  # if X_train['default_unknown'] > 0.5
                return 'Node_8'

In [22]:
df['nodes']=df.apply(tree, axis=1)
df_n= pd.get_dummies(df['nodes'],drop_first=True)
df_2=pd.concat([df, df_n], axis=1)
df_2=df_2.drop(['nodes'],axis=1)

In [23]:
X = df_2.iloc[:,1:]
y = df_2.iloc[:,0]
os = SMOTE(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
columns = X_train.columns
os_data_X,os_data_y=os.fit_sample(X_train, y_train)

In [24]:
nodes=df_n.columns.tolist()
Init = os_data_X.drop(nodes,axis=1).columns.tolist()

## 2.3 Logistic Regression (initial)

In [25]:
from sklearn.linear_model import LogisticRegression
lr0 = LogisticRegression(C=0.001, random_state=1)
lr0.fit(os_data_X[Init], os_data_y)
y_pred0 = lr0.predict(X_test[Init])
print('Misclassified samples: %d' % (y_test != y_pred0).sum())
print(metrics.classification_report(y_test, y_pred0))
print (roc_auc_score(y_test, y_pred0))

Misclassified samples: 2781
              precision    recall  f1-score   support

           0       0.95      0.79      0.86     10981
           1       0.28      0.66      0.40      1376

    accuracy                           0.77     12357
   macro avg       0.62      0.73      0.63     12357
weighted avg       0.88      0.77      0.81     12357

0.7268484557364412


## 2.4 Logistc Rergression with Decision Tree nodes

In [26]:
from sklearn.linear_model import LogisticRegression
lr1 = LogisticRegression(C=0.001, random_state=1)
lr1.fit(os_data_X, os_data_y)
y_pred1 = lr1.predict(X_test)
print('Misclassified samples: %d' % (y_test != y_pred1).sum())
print(metrics.classification_report(y_test, y_pred1))
print (roc_auc_score(y_test, y_pred1))

Misclassified samples: 2465
              precision    recall  f1-score   support

           0       0.95      0.82      0.88     10981
           1       0.31      0.65      0.42      1376

    accuracy                           0.80     12357
   macro avg       0.63      0.73      0.65     12357
weighted avg       0.88      0.80      0.83     12357

0.7345623280592481


## 2.5 Result comparison

In [ ]:
y_pred0b=lr0.predict_proba(X_test[Init])
y_pred1b=lr1.predict_proba(X_test)

In [ ]:
import numpy as np
def lift (test, pred, cardinaility):

    res = pd.DataFrame(np.column_stack((test, pred)),
                       columns=['Target','PR_0', 'PR_1'])

    res['scr_grp'] = pd.qcut(res['PR_0'], cardinaility, labels=False)+1

    crt = pd.crosstab(res.scr_grp, res.Target).reset_index()
    crt = crt.rename(columns= {'Target':'Np',0.0: 'Negatives', 1.0: 'Positives'})

    G = crt['Positives'].sum()
    B = crt['Negatives'].sum()
   
    avg_resp_rate = G/(G+B)

    crt['resp_rate'] = round(crt['Positives']/(crt['Positives']+crt['Negatives']),2)
    crt['lift'] = round((crt['resp_rate']/avg_resp_rate),2)
    crt['rand_resp'] = 1/cardinaility
    crt['cmltv_p'] = round((crt['Positives']).cumsum(),2)
    crt['cmltv_p_perc'] = round(((crt['Positives']/G).cumsum())*100,1)
    crt['cmltv_n'] = round((crt['Negatives']).cumsum(),2)  
    crt['cmltv_n_perc'] = round(((crt['Negatives']/B).cumsum())*100,1)   
    crt['cmltv_rand_p_perc'] = (crt.rand_resp.cumsum())*100
    crt['cmltv_resp_rate'] = round(crt['cmltv_p']/(crt['cmltv_p']+crt['cmltv_n']),2)   
    crt['cmltv_lift'] = round(crt['cmltv_resp_rate']/avg_resp_rate,2)
    crt['KS']=round(crt['cmltv_p_perc']-crt['cmltv_rand_p_perc'],2)
    crt = crt.drop(['rand_resp','cmltv_p','cmltv_n',], axis=1)
    
    print('average response rate: ' , avg_resp_rate)
    return crt

In [ ]:
ModelLift0 = lift(y_test,y_pred0b,10)
ModelLift0

In [ ]:
ModelLift1 = lift(y_test,y_pred1b,10)
ModelLift1